Before you begin, execute this cell to import numpy and packages from the D-Wave Ocean suite, and all necessary functions the gate-model framework you are going to use, whether that is the Forest SDK or Qiskit. In the case of Forest SDK, it also starts the qvm and quilc servers.

In [1]:
%run -i "assignment_helper.py"

Available frameworks:
Forest SDK
Qiskit
D-Wave Ocean


# Quantum Fourier Transform

**Exercise 1** (3 points). The quantum Fourier transform acts on an amplitude-encoded variant of a normalized vector, the quantum state $|x\rangle=\sum_{i=0}^{N-1} x_i |i \rangle$. It maps this state to $|y\rangle=\sum_{k=0}^{N-1} y_k |k \rangle$, where $y_k = \frac{1}{\sqrt{N}}\sum_{j=0}^{N-1}x_je^{\boldsymbol{2\pi i} \frac{jk}{N}}$. If you have a two-dimensional vector to transform, you only need a single qubit, and the transformation reduces to $U |x \rangle = \frac{1}{\sqrt{2}} \left(|0\rangle + e^{\boldsymbol{2\pi i} 0.x_1} |1\rangle\right)$. Implement this circuit. Place your solution in an object called `qft`. 

In [2]:
###
### YOUR CODE HERE
###
import numpy as np
from pyquil import Program, get_qc
from pyquil.gates import *
#from forest_tools import *
qvm_server, quilc_server, fc = init_qvm_and_quilc('/home/local/bin/qvm', '/home/local/bin/quilc')
qc = get_qc('1q-qvm', connection=fc)
π = np.pi
qft = Program()
qft += H(0)

In [3]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**Exercise 2** (1 point). Implement the inverse quantum Fourier transformation on a single qubit. Place your solution in an object called `iqft`. 

In [4]:
###
### YOUR CODE HERE
###
#iqft = Program()
#iqft += PHASE(-π/2, 0)
#iqft += H(0)
iqft = qft.dagger()

In [5]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


# Quantum phase estimation

**Exercise 3** (3 points). Phase estimation pivots on the controlled application of powers of a unitary matrix, followed by an inverse Fourier transformation. Take the $2\times 2$ unitary matrix $\begin{bmatrix}1 & 0 \\0 & i\end{bmatrix}$. This operation has two eigenvectors, $|0\rangle$ and $|1\rangle$. Implement the first part of the quantum phase estimation with two ancilla qubits: add the necessary Hadamard gates on the ancilla qubits and apply $C-U^{2^0}$ and $C-U^{2^1}$. Qubits 0 and 1 should be the ancilla, and qubit 2 contains the eigenvector -- you can assume that this is $|0\rangle$. Place your solution in an object called `circuit`. 

In [6]:
CPHASE12 = Program(CPHASE(π, 1, 2))
CPHASEdag12 = CPHASE12.dagger()

In [8]:
###
### YOUR CODE HERE
###
from grove.alpha.phaseestimation.phase_estimation import controlled
circuit = Program()
circuit += H(0)
circuit += H(1)
circuit.defgate('CS', controlled(np.array([[1, 0], [0, 1j]])))
circuit += ('CS', 1, 2)
#circuit.defgate('CSdag', controlled(np.array([[1, 0], [0, -1j]])))
#circuit += ('CSdag', 1, 2)
circuit += CZ(0, 2)
#circuit += CPHASEdag12

In [9]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j]))

**Exercise 4** (1 point). Apply the same circuit to the other eigenvector. Place your solution in an object called `circuit_2`.

In [12]:
###
### YOUR CODE HERE
###
circuit_2 = Program()
circuit_2 += H(0)
circuit_2 += H(1)
circuit_2 += X(2)
circuit_2.defgate('CS', controlled(np.array([[1, 0], [0, 1j]])))
circuit_2 += ('CS', 1, 2)
circuit_2 += CZ(0, 2)

In [13]:
amplitudes = get_amplitudes(circuit_2)
amplitudes

array([ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,
        0. +0.5j,  0. -0.5j])

In [14]:
amplitudes = get_amplitudes(circuit_2)
assert np.allclose(amplitudes, np.array([ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,
                                          0. +0.5j,  0. -0.5j]))